In [222]:
import random
from abc import ABCMeta, abstractmethod

import numbers
import itertools
from itertools import product

In [1188]:
class Location():
    
    def __init__(self, x=1, y=1):
        self.x = x
        self.y = y

class Percept():
    
    def __init__(self):
        self.stench = False
        self.breeze = False
        self.glitter = False
        self.bump = False
        self.scream = False

In [1189]:
class NaiveAgent():
    def __init__(self):
        self.agent_type = 'naive_agent'
        self.all_actions = ['forward', 'turn_left', 'turn_right', 'grab', 'shoot', 'climb']
        #self.current_action = self.random_action()
        self.legal_actions = self.all_actions
        #self.agent_loc=
        
    def random_action(self):
        import random
        return random.choice(self.legal_actions)

In [1190]:
class WumpusEnvironment():
    
    
    def __init__(self, grid = 4, allow_climb_without_gold= True, pit_prob =0.2):
       
        self.grid = grid
        self.x_start, self.y_start = (1, 1)
        self.x_end, self.y_end = (grid,grid)
        self.pit_prob = pit_prob
        self.allow_climb_without_gold = allow_climb_without_gold
        self.agent_loc = Location(self.x_start, self.y_start)
        self.orientation = 'RIGHT'
        self.agent_alive = True
        self.agent_has_arrow = True
        self.agent_has_gold = False
        self.wumpus_alive = True
        self.make_things()
        self.make_percepts()
        self.full_world()
        self.game_over = False

        
    def make_things(self):
        
        self.cells =list(product((list(range(1,5))),repeat=2))
        self.things =  {key: [] for key in cells}
        self.pits = []

        
        #create pits:
        for x in range(self.x_start, self.x_end + 1):
            for y in range(self.y_start, self.y_end+1):
                if (x != 1) or (y != 1):
                    if random.random() < self.pit_prob:
                        self.pits.append((x, y))
                        self.things[(x,y)].append('pit')
        
        
        #create wumpus
        self.wumpus=(random.randint(self.x_start, self.x_end), random.randint(self.y_start, self.y_end))
        while self.wumpus==(self.x_start,self.y_start):
            self.wumpus=(random.randint(self.x_start, self.x_end), random.randint(self.y_start, self.y_end))
        self.things[self.wumpus].append('wumpus')
    
                    
        #create gold:
        self.gold=(random.randint(self.x_start, self.x_end), random.randint(self.y_start, self.y_end))
        while self.gold==(self.x_start,self.y_start):
            self.gold=(random.randint(self.x_start, self.x_end), random.randint(self.y_start, self.y_end))
        self.things[self.gold].append('gold')
                
        return self.things,self.pits, self.wumpus,self.gold
            
        
        #create percepts
    def same_loc (self,loc1, loc2):
        return loc1[0] == loc2[0] and loc1[1] == loc2[1]
    
    def is_adjacent(self,loc1, loc2):
        x1 = loc1[0]
        x2 = loc2[0]
        y1 = loc1[1]
        y2 = loc2[1]

        if (x1 == x2) and (y1 == (y2 - 1)) or \
           (x1 == x2) and (y1 == (y2 + 1)) or \
           (x1 == (x2 - 1)) and (y1 == y2) or \
           (x1 == (x2 + 1)) and (y1 == y2):
            return True
        return False
    
    def make_percepts(self):
        
        self.percepts = {key: [] for key in self.cells}
        for cell in self.percepts.keys():
            for pit in self.pits:
                if  self.is_adjacent(cell, pit) and 'breeze' not in self.percepts[cell]:
                    self.percepts[cell].append('breeze')


            if self.is_adjacent(cell,self.wumpus)or self.same_loc(cell,self.wumpus):
                self.percepts[cell].append('stench')

            if self.same_loc(cell,self.gold):
                self.percepts[cell].append('glitter')
        return self.percepts
    
    
    def full_world(self):
        import copy
        self.world = copy.deepcopy(self.things)
        for key in self.world.keys():
            if self.percepts[key]:
                self.world[key].extend(self.percepts[key])
        return self.world    
    
    def game_over(self): -> bool
        return self.game_over
        

In [ ]:
def main():
    agent = NaiveAgent()
    env = WumpusEnvironment()
    
    while not env.game_over():
        agent.perceive(env.get_percept())
        env.take_action(agent.next_action)
        print(f"Score: {env.get_score()}")
        print(f"Percept:")
        if env.game_result() == "win"
            print("")

In [1201]:
'''REVIEW SCORING ONCE CONFIRMED BY THE TEACHERS'''


class GamePlay():
    
    def __init__(self,agent):
        self.game_state = WumpusEnvironment()
        self.percepts = self.game_state.percepts
        self.agent_loc = self.game_state.agent_loc
        self.agent_loc.x,self.agent_loc.y=self.agent_loc.x, self.agent_loc.y
        self.grid = self.game_state.grid
        self.orientation = self.game_state.orientation
        self.score=0
        self.game_over = False
        self.agent = agent
        self.agent_alive = self.game_state.agent_alive
        self.agent_has_gold = self.game_state.agent_has_gold
        self.wumpus_location= self.thing_position('wumpus')
        
    def get_percept(self,location):
        return self.percepts[location]
     
    
    #to find location of wumpus and gold:
    def thing_position(self,thing):
        keys=list(self.game_state.things.keys())
        values=list(self.game_state.things.values())

        try:
            values.index([thing])
        except:
            try: 
                for sublist in values:
                    if thing in sublist:
                        position = values.index(sublist)
            except:
                print('exception ocurred')
            #else:
        else:
            position = values.index([thing])
            
        return (keys[position])
    
    
    def action(self,action):
        
        if action == 'forward':
            if self.orientation == 'RIGHT':
                if  self.agent_loc.x < self.grid:
                    self.agent_loc.x += 1
                else:
                    print('Bump!')
            elif self.orientation == 'UP':
                if self.agent_loc.y < self.grid:
                    self.agent_loc.y += 1
                else:
                      print('Bump!')
                        
            elif self.orientation == 'LEFT':
                if self.agent_loc.x > 1:
                    self.agent_loc.x -= 1
                else:
                      print('Bump!')
                        
            elif self.orientation == 'DOWN':
                if self.agent_loc.y > 1:
                    self.agent_loc.y -= 1
                else:
                     print('Bump!')
                            
       
        if action == 'turn_left':
            if self.orientation == 'RIGHT':
                self.orientation = 'UP'
            elif self.orientation == 'UP':
                self.orientation = 'LEFT'
            elif self.orientation == 'LEFT':
                self.orientation = 'DOWN'
            elif self.orientation == 'DOWN':
                self.orientation = 'RIGHT'

        if action == 'turn_right':
            if self.orientation == 'RIGHT':
                self.orientation = 'DOWN'
            elif self.orientation == 'UP':
                self.orientation = 'RIGHT'
            elif self.orientation == 'LEFT':
                self.orientation = 'UP'
            elif self.orientation == 'DOWN':
                self.orientation = 'LEFT'

        if action == 'grab':
    #change to accomodate list
            if not self.game_state.agent_has_gold and self.game_state.things[self.thing_position('gold')]== 'gold':
                self.game_state.agent_has_gold = True
                
        if action == 'shoot':
            
            if  self.game_state.agent_has_arrow:
                self.game_state.agent_has_arrow = False
                
                
               
                if self.game_state.wumpus_alive:
                    self.wumpus_location = self.thing_position('wumpus')
                    print(f'wumpus location: {self.wumpus_location}')
                    if (((self.game_state.orientation == 'RIGHT') and
                         (self.agent_loc.x < self.wumpus_location[0]) and
                         (self.agent_loc.y == self.wumpus_location[1])) or
                        
                        ((self.game_state.orientation == 'UP') and
                         (self.agent_loc.x == self.wumpus_location[0]) and
                         (self.agent_loc.y < self.wumpus_location[1])) or
                        
                        ((self.game_state.orientation == 'LEFT') and
                         (self.agent_loc.x > self.wumpus_location[0]) and
                         (self.agent_loc.y == self.wumpus_location[1])) or
                        
                        ((self.game_state.orientation == 'DOWN') and
                         (self.agent_loc.x == self.wumpus_location[0]) and
                         (self.agent_loc.y > self.wumpus_location[1]))):
                        self.game_state.wumpus_alive = False
                        print("scream!")
                        
            else:
                print('No arrows!')
                
         
                

        if action == 'climb':
            if self.agent_loc.x == self.game_state.x_start and self.agent_loc.y == self.game_state.y_start and\
            self.agent_has_gold:
                
                self.end_of_round()
            else:
                print('Illegal action! You cannot climb without gold or outside of start square!')
    
    
    def play_game(self, agent):
        #since agent is random, we need a condition to break the while loop:
        
        if self.agent.agent_type=='naive_agent':
            max_moves = 40
            i=0
            while not self.game_over and i<=max_moves:

                self.new_move=self.agent.random_action()
                self.action(self.new_move)
                self.score-=1
                percepts = self.game_state.percepts[(self.agent_loc.x,self.agent_loc.y)]
                print(f"agent location: {(self.agent_loc.x,self.agent_loc.y)}"), print(f"chosen_move: {self.new_move}"),\
                print(f"percepts:{percepts}")
                i+=1
#initialize wumpus-LOCATION OUTSIDE OF SHOOTING OR IT WON'T EXIST
                if ((self.agent_loc.x,self.agent_loc.y)==self.wumpus_location and self.game_state.wumpus_alive) or \
                    (self.agent_loc.x,self.agent_loc.y) in self.game_state.pits:
                    self.agent_alive = False
                    self.game_over = True
                    print(f"You've encountered a {self.game_state.things[(self.agent_loc.x,self.agent_loc.y)]}!")
                    self.end_of_round()
        else:
            while not self.game_over :
                self.new_move=self.agent.random_action()
                self.action(self.new_move)

                self.agent.random_action()
                self.score-=1
                percepts = self.game_state.percepts[self.agent_loc.location]

                if (self.agent_location==self.wumpus_location and self.game_state.wumpus_alive) or \
                (self.agent_location in self.game_state.pits):
                    self.agent_alive = False
                    print(f" NOT NAIVE: You've encountered a {self.game_state.things[self.agent_location]}!")
                    self.end_of_round()
            
    
    def end_of_round(self):
        #ALSO NEEDS TO CLIMB OUT:
        if self.agent_alive and self.agent_has_gold:
            print("Congratulations! You won!")        
        else:
            print('Wa-Wa-Wa-Waaa! You lose!')
        print(self.score)
        self.game_over = True
        
        new_game= input("Do you want to play again? Y or N\n")
        if new_game in["Y","y"]:
            return self.__init__(self.agent)
        else:
            print("Thank you for playing!")
                
        
''' 
     figure out a way around bump and scream percepts other than printing'''   


    

' \n     figure out a way around bump and scream percepts other than printing'

In [1202]:
new_game = GamePlay(NaiveAgent())

In [1203]:
new_game.play_game(NaiveAgent())

wumpus location: (2, 1)
scream!
agent location: (1, 1)
chosen_move: shoot
percepts:['breeze', 'stench']
agent location: (2, 1)
chosen_move: forward
percepts:['stench']
You've encountered a ['pit', 'wumpus']!
Wa-Wa-Wa-Waaa! You lose!
-2
Do you want to play again? Y or N
Y
agent location: (1, 1)
chosen_move: turn_right
percepts:['breeze']
agent location: (1, 1)
chosen_move: turn_right
percepts:['breeze']
agent location: (1, 1)
chosen_move: turn_right
percepts:['breeze']
agent location: (1, 2)
chosen_move: forward
percepts:[]
You've encountered a ['pit']!
Wa-Wa-Wa-Waaa! You lose!
-4
Do you want to play again? Y or N
Y
agent location: (1, 1)
chosen_move: turn_left
percepts:[]
agent location: (1, 1)
chosen_move: grab
percepts:[]
wumpus location: (4, 3)
agent location: (1, 1)
chosen_move: shoot
percepts:[]
Illegal action! You cannot climb without gold or outside of start square!
agent location: (1, 1)
chosen_move: climb
percepts:[]
No arrows!
agent location: (1, 1)
chosen_move: shoot
percept

In [932]:
new.random_action

<bound method NaiveAgent.random_action of <__main__.NaiveAgent object at 0x7ff061df44f0>>

In [933]:
act=['forward', 'turn_left', 'turn_right', 'grab', 'shoot', 'climb']
print(random.choice(act))

grab


In [1170]:
def action(action, grid=4):
        
        if action == 'forward':
            if orientation == 'RIGHT':
                if  agent_loc.x < grid:
                    agent_loc.x += 1
                    print(agent_loc.x)
                else:
                    print('Bump!')
            elif orientation == 'UP':
                if agent_loc.y < grid:
                    agent_loc.y += 1
                else:
                      print('Bump!')
                        
            elif orientation == 'LEFT':
                if agent_loc.x > 1:
                    agent_loc.x -= 1
                else:
                      print('Bump!')
                        
            elif orientation == 'DOWN':
                if agent_loc.y > 1:
                    agent_loc.y -= 1
                else:
                     print('Bump!')
        print(agent_loc.x, agent_loc.location)

In [780]:

def find_posiition(thing):
    
    keys=list(self.game_state.things.keys())
    values=list(self.game_state.things.values())

    try:
        values.index([thing])
    except:
        try: 
            for sublist in values:
                if thing in sublist:
                    position = values.index(sublist)
        except:

            print('exception ocurred')
    else:
        position = values.index([thing])

    return keys[position]

In [721]:
new_game = GamePlay()
new_game.game_state.things

{(1, 1): [],
 (1, 2): ['gold'],
 (1, 3): [],
 (1, 4): [],
 (2, 1): [],
 (2, 2): ['pit'],
 (2, 3): [],
 (2, 4): ['pit'],
 (3, 1): [],
 (3, 2): [],
 (3, 3): ['pit'],
 (3, 4): [],
 (4, 1): ['pit'],
 (4, 2): [],
 (4, 3): ['wumpus'],
 (4, 4): []}

In [64]:
class Wumpus_Environment():
    
    def __init__(self, width=4, height=4):
        super().__init__()
        self.width = width
        self.height = height
        self.x_start, self.y_start = (1, 1)
        self.x_end, self.y_end = (self.width, self.height)
       
        
    def new_game(self):
        self.agent_loc = (1, 1)
        self.agent_orientation = RIGHT
        self.agent_alive = True
        self.agent_has_arrow = True
        self.agent_has_gold = False
        self.wumpus_alive = True
        
    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        self.things = []
        self.agents = []
        if isinstance(location, numbers.Number):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]
        
   # def create_env():
    __all__ = ["Number", "Complex", "Real", "Rational", "Integral"]
    class Thing:
        """This represents any physical object that can appear in an Environment.
        You subclass Thing to get the things you want. Each thing can have a
        .__name__  slot (used for output only)."""

        def __repr__(self):
            return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

        def is_alive(self):
            """Things that are 'alive' should return true."""
            return hasattr(self, 'alive') and self.alive

        def show_state(self):
            """Display the agent's internal state. Subclasses should override."""
            print("I don't know how to show_state.")

        def display(self, canvas, x, y, width, height):
            """Display an image of this Thing on the canvas."""
            # Do we need this?
            pass
        
        def add_thing(self, thing, location=None):
        
            if not isinstance(thing, Thing):
                thing = Agent(thing)
            if thing in self.things:
                print("Can't add the same thing twice")
            else:
                thing.location = location if location is not None else self.default_location(thing)
                self.things.append(thing)
                if isinstance(thing, Agent):
                    thing.performance = 0
                    self.agents.append(thing)


        def init_world(self, program):
            """Spawn items in the world based on probabilities from the book"""

            "WALLS"
            self.add_walls()

            "PITS"
            for x in range(self.x_start, self.x_end):
                for y in range(self.y_start, self.y_end):
                    if random.random() < self.pit_probability:
                        self.add_thing(Pit(), (x, y), True)
                        self.add_thing(Breeze(), (x - 1, y), True)
                        self.add_thing(Breeze(), (x, y - 1), True)
                        self.add_thing(Breeze(), (x + 1, y), True)
                        self.add_thing(Breeze(), (x, y + 1), True)

            "WUMPUS"
            w_x, w_y = self.random_location_inbounds(exclude=(1, 1))
            self.add_thing(Wumpus(lambda x: ""), (w_x, w_y), True)
            self.add_thing(Stench(), (w_x - 1, w_y), True)
            self.add_thing(Stench(), (w_x + 1, w_y), True)
            self.add_thing(Stench(), (w_x, w_y - 1), True)
            self.add_thing(Stench(), (w_x, w_y + 1), True)

            "GOLD"
            self.add_thing(Gold(), self.random_location_inbounds(exclude=(1, 1)), True)

            "AGENT"
            self.add_thing(Explorer(program), (1, 1), True)

        def get_world(self, show_walls=False):
            """Return the items in the world"""
            result = []
            x_start, y_start = (1, 1) 

            if show_walls:
                x_end, y_end = self.width, self.height
            else:
                x_end, y_end = self.width - 1, self.height - 1

            for x in range(x_start, x_end):
                row = []
                for y in range(y_start, y_end):
                    row.append(self.list_things_at((x, y)))
                result.append(row)
            return result
    
        

In [41]:
    


num = Number()

In [68]:
world=Wumpus_Environment()
get_world(world)

[[[], []], [[], []]]

In [5]:
    def is_adjacent(loc1, loc2):
    

        x1 = loc1.x
        x2 = loc2.x
        y1 = loc1.y
        y2 = loc2.y

        if (x1 == x2) and (y1 == (y2 - 1)) or \
           (x1 == x2) and (y1 == (y2 + 1)) or \
           (x1 == (x2 - 1)) and (y1 == y2) or \
           (x1 == (x2 + 1)) and (y1 == y2):
            return True

        return False